# **Exercicio Duelo de Modelos 4**

Nesta tarefa, vocês irão criar o seu próprio duelo de modelos, com o objetivo de superar os resultados apresentados em aula. O desafio é alcançar um desempenho superior ao que obtivemos, e para isso, será necessário aplicar todas as melhorias que vocês aprenderam ao longo dos módulos, utilizando a base de dados do Titanic.

**1. Escolha do Modelo:**
Selecione um dos modelos que foram explorados nos duelos de modelos ao longo do curso. Pode ser SVM, Random Forest, XGBoost, ou qualquer outro que tenhamos abordado.

**2. Aperfeiçoamento:**
**Aplique as técnicas que aprendemos para melhorar o desempenho do seu modelo:**

**Hiperparâmetros:** Utilize GridSearchCV ou RandomSearchCV para encontrar os melhores parâmetros.

**Cross Validation:** Avalie a robustez do modelo utilizando validação cruzada para garantir que ele generaliza bem.

**Balanceamento de Classes:** Se o seu modelo lida com problemas de classes desbalanceadas, explore técnicas como SMOTE, undersampling ou oversampling.

**Padronização e Normalização:** Lembre-se de padronizar os dados, especialmente se for usar modelos que são sensíveis à escala das variáveis.

**3. Submissão no Kaggle:**
Treine o seu modelo com os dados de treino e gere as previsões para os dados de teste. Lembre-se de que o conjunto de teste não possui a variável alvo (y_test), pois a avaliação será feita com base nas submissões no Kaggle.
Submeta suas previsões na competição do Titanic no Kaggle.

**4. Entrega:**
Envie o código que você desenvolveu, detalhando cada etapa do seu processo de modelagem, explicando as escolhas feitas e como essas ajudaram a melhorar o modelo.

Junto com o código, envie um print do seu score obtido na plataforma do Kaggle. Esse score será a sua métrica final de avaliação, mostrando como o seu modelo se compara com os demais.

**5. Competição Saudável:**
A ideia é trazer um senso de competição saudável, então não vale replicar exatamente o que fizemos na aula! Inove, explore novas combinações de parâmetros e técnicas, e mostre do que é capaz. O importante é exercitar o pensamento crítico e a capacidade de experimentar.

**Dicas Finais:**

Seja criativo e tenha um olhar crítico sobre o que pode ser melhorado.
Teste diferentes abordagens e não se prenda a um único caminho.
Lembre-se de que, mais do que alcançar o melhor score, o objetivo é aprender e aplicar o conhecimento de forma prática e eficaz.
Boa sorte! Estamos ansiosos para ver como cada um de vocês vai se sair nesse desafio e quais insights irão surgir dessa competição!

Ao final dessa atividade vocês terão participado da primeira competição publica de ciência de dados de vocês = )




In [6]:
# ===================================================
# 1️⃣ Imports e Configurações Globais
# ===================================================

import os, re, warnings, random
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool

ModuleNotFoundError: No module named 'catboost'

1️⃣ Imports e Configurações Globais

O que tem dentro:

os, re, warnings, random: utilitários do Python para manipulação de arquivos, expressões regulares, warnings e geração de números aleatórios.

numpy, pandas: bibliotecas essenciais para cálculo numérico e manipulação de tabelas (DataFrames).

sklearn.model_selection.StratifiedKFold: para criar folds estratificados na validação cruzada.

sklearn.metrics.accuracy_score: para medir a acurácia do modelo.

catboost.CatBoostClassifier, Pool: modelo principal e estrutura de dados específica para CatBoost.

Função:

Permite ler e manipular os dados (pandas), fazer cálculos e operações numéricas (numpy), criar validação cruzada estratificada (StratifiedKFold), medir performance (accuracy_score) e treinar o modelo (CatBoost).

Por que escolhi:

CatBoost é ótimo para dados mistos (numéricos + categóricos) e geralmente supera XGBoost/LightGBM sem precisar de muito encoding manual.

StratifiedKFold evita que folds tenham distribuição desigual de sobreviventes.

warnings e np.set_printoptions garantem que o notebook seja mais limpo e legível.

Configurações globais:

RANDOM_STATE: para reprodutibilidade.

N_SPLITS: número de folds para CV.

TE_SMOOTHING: suavização para target encoding, evita overfitting.

EARLY_STOPPING_ROUNDS: para parar o treinamento se não houver melhora.

PSEUDO_LABEL, PL_POS_THR, PL_NEG_THR: controlam pseudo-labeling

In [ ]:
# Configurações gerais
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

RANDOM_STATE = 42
N_SPLITS = 5
TE_SMOOTHING = 20.0
EARLY_STOPPING_ROUNDS = 200
PSEUDO_LABEL = True          # True para usar pseudo-labeling, False para não usar
PL_POS_THR, PL_NEG_THR = 0.98, 0.02

In [ ]:
# ===================================================
# 2️⃣ Funções utilitárias
# ===================================================

def seed_everything(seed=42):
    """Fixar seeds para reprodutibilidade"""
    random.seed(seed)
    np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass

def detect_input_dir():
    """Detecta a pasta onde os arquivos train/test CSV estão localizados"""
    for p in ["../input/titanic", "../input/Titanic", "./"]:
        if os.path.exists(os.path.join(p, "train.csv")):
            return p
    return "./"

def extract_title(name: str) -> str:
    """Extrai título do nome do passageiro"""
    if pd.isna(name): return "Unknown"
    m = re.search(r",\s*([^\.]+)\.", name)
    return m.group(1).strip() if m else "Unknown"

def clean_ticket_prefix(s: str) -> str:
    """Limpa e padroniza prefixo do ticket"""
    if pd.isna(s): return "NONE"
    s = str(s).upper()
    s = re.sub(r"[./]", "", s)
    s = re.sub(r"\s+", "", s)
    s = re.sub(r"\d+", "", s)
    return s if s else "NONE"

2️⃣ Funções utilitárias

O que tem dentro:

seed_everything(): fixa sementes de aleatoriedade no Python, numpy e PyTorch (se disponível) para resultados reproduzíveis.

detect_input_dir(): localiza automaticamente onde os arquivos train.csv e test.csv estão.

extract_title(name): extrai o título do passageiro do campo Name.

clean_ticket_prefix(s): limpa e padroniza prefixos de tickets (ex.: "A/5 21171" → "A").

Função:

Garantir consistência nos experimentos (seed_everything).

Facilitar leitura de dados sem precisar alterar paths (detect_input_dir).

Criar features relevantes a partir de campos textuais (extract_title e clean_ticket_prefix).

Por que escolhi:

A análise do Titanic mostra que títulos (Mr, Mrs, Master) e prefixos de tickets podem refletir status social ou grupos de viagem, impactando sobrevivência.

Limpeza de strings evita categorias inconsistentes e reduz cardinalidade.

In [ ]:
# ===================================================
# 3️⃣ Função de Preprocessamento
# ===================================================

def preprocess_full(df: pd.DataFrame, train_len: int) -> pd.DataFrame:
    """
    Criação de features derivadas, imputação de valores ausentes,
    winsorização suave e normalização.
    """
    out = df.copy()
    
    # Normalize text columns
    out["Sex"] = out["Sex"].astype(str).str.lower()
    out["Embarked"] = out["Embarked"].astype(str).str.upper()

    # Name-based features
    out["Title"] = out["Name"].apply(extract_title)
    title_counts_train = out.iloc[:train_len]["Title"].value_counts()
    rare_titles = title_counts_train[title_counts_train < 10].index
    out["Title"] = out["Title"].where(~out["Title"].isin(rare_titles), "Rare")
    out["LastName"] = out["Name"].fillna("").apply(lambda s: s.split(",")[0].strip() if "," in s else "Unknown")

    # Family / group
    out["FamilySize"] = out["SibSp"].fillna(0) + out["Parch"].fillna(0) + 1
    out["IsAlone"] = (out["FamilySize"] == 1).astype(int)

    # Ticket features
    out["TicketPrefix"] = out["Ticket"].apply(clean_ticket_prefix)
    ticket_counts = out["Ticket"].fillna("NA").value_counts()
    out["TicketGroupSize"] = out["Ticket"].fillna("NA").map(ticket_counts).fillna(1).astype(int)

    # Cabin features
    out["HasCabin"] = (~out["Cabin"].isna()).astype(int)
    out["CabinDeck"] = out["Cabin"].fillna("M").astype(str).str[0].str.upper()
    out["CabinMulti"] = out["Cabin"].astype(str).str.contains(r"\s").astype(int)

    # Embarked
    out["Embarked"] = out["Embarked"].where(out["Embarked"].isin(["S","C","Q"]), np.nan)
    mode_val = out.iloc[:train_len]["Embarked"].mode()
    mode_val = mode_val.iloc[0] if not mode_val.empty else "S"
    out["Embarked"] = out["Embarked"].fillna(mode_val)

    # Numeric types
    out["Pclass"] = out["Pclass"].astype(int)
    out["Fare"] = out["Fare"].astype(float)
    out["Age"] = out["Age"].astype(float)

    # Imputations
    pclass_med = out.iloc[:train_len].groupby("Pclass")["Fare"].median()
    out["Fare"] = out.apply(lambda r: pclass_med[r["Pclass"]] if pd.isna(r["Fare"]) else r["Fare"], axis=1)

    grp_med = out.iloc[:train_len].groupby(["Title","Pclass","Sex"])["Age"].median()
    def impute_age(r):
        if not pd.isna(r["Age"]): return r["Age"]
        key = (r["Title"], r["Pclass"], r["Sex"])
        return grp_med.get(key, out.iloc[:train_len]["Age"].median())
    out["Age"] = out.apply(impute_age, axis=1)

    # Derived features
    out["FarePerPerson"] = out["Fare"] / out["TicketGroupSize"]
    out["IsChild"] = (out["Age"] < 16).astype(int)
    out["IsMother"] = ((out["Sex"]=="female") & (out["Parch"]>0) & (out["Title"]=="Mrs")).astype(int)

    # Winsorization suave
    out["Fare"] = out["Fare"].clip(0, np.nanpercentile(out["Fare"], 99))
    out["FarePerPerson"] = out["FarePerPerson"].clip(0, np.nanpercentile(out["FarePerPerson"], 99))
    out["Age"] = out["Age"].clip(0, 80)

    return out


3️⃣ Função de Preprocessamento

O que tem dentro:

Normalização de colunas categóricas (Sex, Embarked) para evitar inconsistências.

Features derivadas do nome: Title, LastName.

Features familiares: FamilySize, IsAlone.

Features de ticket: TicketPrefix, TicketGroupSize.

Features de cabine: HasCabin, CabinDeck, CabinMulti.

Imputação de valores ausentes para Fare e Age.

Features derivadas: FarePerPerson, IsChild, IsMother.

Winsorização suave para limitar outliers em Fare, Age e FarePerPerson.

Função:

Transformar dados brutos em features numéricas e categóricas limpas, que podem ser usadas diretamente pelo CatBoost.

Criar indicadores que capturam padrões sociais/familiares importantes para sobrevivência.

Por que escolhi:

O CatBoost lida bem com categóricas, mas features bem pensadas aumentam muito a performance.

Winsorização reduz impacto de outliers extremos, comum em Fare.

Imputações baseadas em grupos (Title + Pclass + Sex) respeitam a distribuição dos dados e evitam vazamento.

In [ ]:
# ===================================================
# 4️⃣ Função K-Fold Target Encoding
# ===================================================

def kfold_target_mean_encode(X_tr: pd.DataFrame, y_tr: pd.Series,
                             X_te: pd.DataFrame, col: str,
                             n_splits=5, smoothing=20.0, random_state=42) -> Tuple[np.ndarray, np.ndarray]:
    """
    Target Encoding seguro: calcula média da variável target por categoria
    usando K-Fold para evitar vazamento de dados.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    prior = float(y_tr.mean())

    X_tr_col = X_tr[col].reset_index(drop=True)
    y_tr_ser = pd.Series(y_tr).reset_index(drop=True)
    X_te_col = X_te[col].reset_index(drop=True)

    oof = np.zeros(len(X_tr_col), dtype=float)
    for tr_idx, va_idx in skf.split(np.zeros(len(y_tr_ser)), y_tr_ser):
        df_fold = pd.DataFrame({col: X_tr_col.iloc[tr_idx], "y": y_tr_ser.iloc[tr_idx]})
        stats = df_fold.groupby(col)["y"].agg(["mean","count"])
        smooth = (stats["mean"]*stats["count"] + prior*smoothing) / (stats["count"] + smoothing)
        oof[va_idx] = X_tr_col.iloc[va_idx].map(smooth).fillna(prior).values

    stats_full = pd.DataFrame({col: X_tr_col.values, "y": y_tr_ser.values}).groupby(col)["y"].agg(["mean","count"])
    smooth_full = (stats_full["mean"]*stats_full["count"] + prior*smoothing) / (stats_full["count"] + smoothing)
    te = X_te_col.map(smooth_full).fillna(prior).values

    return oof, te


4️⃣ Função K-Fold Target Encoding

O que tem dentro:

Codificação de variáveis categóricas de alta cardinalidade (LastName, TicketPrefix) usando a média da target (Survived) dentro de cada categoria.

Uso de Stratified K-Fold para garantir que a média seja calculada sem vazamento de dados.

Suavização para reduzir overfitting.

Função:

Transformar categorias com muitos valores únicos em uma feature numérica correlacionada à target.

Evita vazamento de informação entre folds (ou seja, a média do grupo não “vaza” a informação do fold de validação).

Por que escolhi:

Variáveis como LastName ou TicketPrefix são muito específicas e não podem ser one-hot encoded.

K-Fold target encoding com smoothing é a técnica segura mais eficiente para isso.

In [ ]:
# ===================================================
# 5️⃣ Carregamento dos Dados e Preprocessamento
# ===================================================

seed_everything(RANDOM_STATE)
INPUT_DIR = detect_input_dir()

train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
test  = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))

y = train["Survived"].astype(int)
train_nolabel = train.drop(columns=["Survived"])
full = pd.concat([train_nolabel, test], axis=0, ignore_index=True)

full = preprocess_full(full, train_len=len(train))


5️⃣ Carregamento dos Dados e Preprocessamento

O que tem dentro:

Leitura de train.csv e test.csv.

Separação de y (target) e features.

Concatenar train + test para pré-processar juntos (evita inconsistência de categorias).

Chamada da função preprocess_full.

Função:

Preparar dataset completo para feature engineering, mantendo separação correta entre treino e teste.

Por que escolhi:

Processar juntos evita que categorias novas no teste não sejam tratadas.

Mantém consistência para target encoding e outras features derivadas.

In [ ]:
# ===================================================
# 6️⃣ Target Encoding de Variáveis Categóricas de Alta Cardinalidade
# ===================================================

base_cat = ["Sex","Embarked","Title","CabinDeck","TicketPrefix"]
base_num = ["Pclass","Age","SibSp","Parch","Fare","FamilySize","IsAlone",
            "IsChild","IsMother","HasCabin","CabinMulti","TicketGroupSize","FarePerPerson"]

te_cols = ["LastName","TicketPrefix","CabinDeck","Title"]
n_train = len(train)
train_idx = np.arange(n_train)
test_idx  = np.arange(n_train, len(full))

X_all = full.copy()
te_train_parts, te_test_parts = {}, {}
for c in te_cols:
    oof_c, test_c = kfold_target_mean_encode(
        X_all.loc[train_idx,[c]], y, X_all.loc[test_idx,[c]],
        col=c, n_splits=N_SPLITS, smoothing=TE_SMOOTHING, random_state=RANDOM_STATE
    )
    te_train_parts[c+"_TE"] = oof_c
    te_test_parts[c+"_TE"] = test_c

for newc, arr in te_train_parts.items(): X_all.loc[train_idx, newc] = arr
for newc, arr in te_test_parts.items():  X_all.loc[test_idx, newc] = arr

use_cols = base_cat + base_num + [c+"_TE" for c in te_cols]
X_train = X_all.loc[train_idx, use_cols].reset_index(drop=True)
X_test  = X_all.loc[test_idx,  use_cols].reset_index(drop=True)


6️⃣ Target Encoding de Variáveis de Alta Cardinalidade

O que tem dentro:

Lista de variáveis a encodar (te_cols).

Loop que aplica kfold_target_mean_encode em cada coluna.

Adiciona colunas codificadas (_TE) ao dataframe.

Função:

Transformar variáveis categóricas complexas em features numéricas seguras.

Preparar dataset pronto para o CatBoost.

Por que escolhi:

Evita overfitting e vazamento, enquanto permite aproveitar informações valiosas de categorias raras.

In [ ]:
# ===================================================
# 7️⃣ Treinamento CatBoost com CV
# ===================================================

cat_idx = [X_train.columns.get_loc(c) for c in base_cat]  # Indices das colunas categóricas
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

oof = np.zeros(len(X_train))
test_pred_folds = np.zeros((N_SPLITS, len(X_test)))
fold_acc = []

for fold, (tr, va) in enumerate(skf.split(X_train, y), 1):
    X_tr, X_va = X_train.iloc[tr], X_train.iloc[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    est = CatBoostClassifier(
        depth=6, learning_rate=0.05, iterations=3000, l2_leaf_reg=3.0,
        loss_function="Logloss", eval_metric="Accuracy",
        random_seed=RANDOM_STATE+fold, od_type="Iter", od_wait=EARLY_STOPPING_ROUNDS, verbose=False
    )
    tr_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
    va_pool = Pool(X_va, y_va, cat_features=cat_idx)
    te_pool = Pool(X_test, cat_features=cat_idx)

    est.fit(tr_pool, eval_set=va_pool, use_best_model=True)
    va_p = est.predict_proba(va_pool)[:,1]
    te_p = est.predict_proba(te_pool)[:,1]

    oof[va] = va_p
    test_pred_folds[fold-1,:] = te_p
    fold_acc.append(accuracy_score(y_va, (va_p>=0.5).astype(int)))

print("CatBoost CV Acc per fold:", [f"{a:.4f}" for a in fold_acc], "| Mean:", f"{np.mean(fold_acc):.4f}")


7️⃣ Treinamento CatBoost com Cross-Validation

O que tem dentro:

Criação de folds estratificados.

Treinamento do CatBoost em cada fold.

Armazenamento das previsões OOF (out-of-fold) e previsão para teste.

Cálculo de acurácia em cada fold.

Função:

Treinar modelo robusto com validação cruzada.

Obter estimativas OOF para tuning de threshold e análise de performance.

Por que escolhi:

CatBoost é rápido e eficiente para dados mistos.

CV estratificado garante avaliação justa e reduz variância na validação.

In [ ]:
# ===================================================
# 8️⃣ Threshold tuning OOF
# ===================================================

ths = np.linspace(0.3,0.7,401)
best_t, best_acc = max(((t, accuracy_score(y,(oof>=t).astype(int))) for t in ths), key=lambda x:x[1])
print(f"OOF Accuracy = {best_acc:.4f} at threshold = {best_t:.3f}")

test_prob = test_pred_folds.mean(axis=0)
pred = (test_prob >= best_t).astype(int)


8️⃣ Threshold tuning OOF

O que tem dentro:

Varredura de thresholds entre 0.3 e 0.7.

Seleção do threshold que maximiza acurácia OOF.

Aplicação do threshold ao teste.

Função:

Ajustar ponto de corte para transformar probabilidades em classes (0/1) de forma otimizada.

Por que escolhi:

O threshold default (0.5) pode não ser ideal devido a desbalanceamento.

Melhora acurácia sem mexer no modelo.

In [ ]:
# ===================================================
# 9️⃣ Pseudo-labeling opcional (safe)
# ===================================================

if PSEUDO_LABEL:
    high_pos = np.where(test_prob >= PL_POS_THR)[0]
    high_neg = np.where(test_prob <= PL_NEG_THR)[0]
    sel = np.concatenate([high_pos, high_neg])

    if sel.size > 0:
        X_aug = pd.concat([X_train, X_test.iloc[sel]], axis=0, ignore_index=True)
        y_aug = pd.concat([y.reset_index(drop=True),
                           pd.Series((test_prob[sel] >= PL_POS_THR).astype(int))],
                          axis=0).reset_index(drop=True)

        oof2 = np.zeros(len(X_train))
        test_pred_folds2 = np.zeros((N_SPLITS,len(X_test)))
        fold_acc2 = []

        for fold, (tr, va) in enumerate(skf.split(X_train,y),1):
            tr_idx_aug = np.concatenate([tr, n_train+np.arange(len(sel))])
            X_tr2, y_tr2 = X_aug.iloc[tr_idx_aug], y_aug.iloc[tr_idx_aug]
            X_va2, y_va2 = X_train.iloc[va], y.iloc[va]

            est2 = CatBoostClassifier(
                depth=6, learning_rate=0.05, iterations=3000, l2_leaf_reg=3.0,
                loss_function="Logloss", eval_metric="Accuracy",
                random_seed=RANDOM_STATE+123+fold, od_type="Iter", od_wait=EARLY_STOPPING_ROUNDS, verbose=False
            )
            tr_pool2 = Pool(X_tr2, y_tr2, cat_features=cat_idx)
            va_pool2 = Pool(X_va2, y_va2, cat_features=cat_idx)
            te_pool2 = Pool(X_test, cat_features=cat_idx)

            est2.fit(tr_pool2, eval_set=va_pool2, use_best_model=True)
            va_p2 = est2.predict_proba(va_pool2)[:,1]
            te_p2 = est2.predict_proba(te_pool2)[:,1]

            oof2[va] = va_p2
            test_pred_folds2[fold-1,:] = te_p2
            fold_acc2.append(accuracy_score(y_va2, (va_p2>=0.5).astype(int)))

        print("PL CatBoost CV Acc per fold:", [f"{a:.4f}" for a in fold_acc2], "| Mean:", f"{np.mean(fold_acc2):.4f}")
        ths2 = np.linspace(0.3,0.7,401)
        best_t2, best_acc2 = max(((t, accuracy_score(y,(oof2>=t).astype(int))) for t in ths2), key=lambda x:x[1])
        print(f"[PL] OOF Accuracy = {best_acc2:.4f} at threshold = {best_t2:.3f}")

        test_prob = test_pred_folds2.mean(axis=0)
        pred = (test_prob >= best_t2).astype(int)
    else:
        print("No high-confidence test samples for pseudo-labeling; skipping.")


9️⃣ Pseudo-labeling opcional

O que tem dentro:

Seleção de amostras de teste com alta confiança (>=0.98 ou <=0.02).

Adição dessas amostras ao treino com labels previstos.

Retraining do CatBoost sem recalcular target encoding.

Função:

Aumentar o dataset de treino com exemplos confiáveis do teste, para potencial melhoria de performance.

Por que escolhi:

Técnica leve e segura, só uma rodada, evita overfitting.

Útil em competições com leaderboard.

In [ ]:
# ===================================================
# 🔟 Submission
# ===================================================

sub = pd.DataFrame({"PassengerId": test["PassengerId"].values, "Survived": pred})
sub.to_csv("submission.csv", index=False)
print("Saved: submission.csv", sub.shape)


🔟 Submission

O que tem dentro:

Criação do dataframe de submissão com PassengerId e Survived.

Salvamento em CSV.

Função:

Gerar arquivo pronto para submissão no Kaggle.

Por que escolhi:

Padrão obrigatório da competição Titanic.

Explicação geral:
Este notebook implementa um pipeline robusto para o desafio do Titanic, com foco em evitar vazamento de dados (leakage) e maximizar acurácia de validação.

Preprocessamento: Criação de features relevantes a partir de nomes, tickets, cabine e relações familiares.

Target Encoding: Codificação de variáveis categóricas de alta cardinalidade de forma segura via K-Fold (sem vazamento).

Treinamento: CatBoost com validação cruzada e early stopping.

Threshold tuning: Ajuste do threshold de probabilidade para melhorar a acurácia OOF.

Pseudo-labeling (opcional): Inclusão de amostras de teste de alta confiança para potencial ganho de performance.

Submission: Geração do arquivo submission.csv pronto para Kaggle.